# Data Import

In [34]:
import pandas as pd
import os
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from geopy.distance import geodesic

from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import mean_absolute_error

from sklearn.neighbors import LocalOutlierFactor

from sklearn.ensemble import IsolationForest

from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Input, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

from joblib import dump

from sklearn.decomposition import PCA

from mpl_toolkits.mplot3d import Axes3D

import plotly.graph_objs as go

import dask.dataframe as dd

import joblib

import gc

import pyarrow

import time

In [2]:
dataset_path = '/kaggle/input/sncb-data-augumentation/augumented_cleaned_ar41_for_ulb.csv'

# # Define the columns that you want to load
# columns_to_load = [
#     'RS_E_InAirTemp_PC1', 'RS_E_InAirTemp_PC2', 
#     'RS_E_WatTemp_PC1', 'RS_E_WatTemp_PC2', 
#     'RS_T_OilTemp_PC1', 'RS_T_OilTemp_PC2',
#     'RS_E_OilPress_PC2', 'RS_E_OilPress_PC2',
#     'RS_E_RPM_PC1', 'RS_E_RPM_PC1',
#     'Speed', 'lat', 'lon', 'mapped_veh_id', 
#     'hour', 'pm10', 'temp_celsius',
#     'weather_main', 'date', 'timestamps_UTC'  # Include 'date' to convert to 'weekday' later
# ]

# Check if the file exists before trying to read it
if os.path.exists(dataset_path):
    # Read the specified columns of the CSV file into a DataFrame
    data = pd.read_csv(dataset_path)

    # Display the basic information and the first few rows of the dataframe
    data_info = data.info()
    data_head = data.head()

    # If you want to print the information to the console
    print("Dataframe Info:")
    print(data_info)
    print("\nFirst Few Rows of Data:")
    print(data_head)
else:
    print(f"The file {dataset_path} does not exist.")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17677337 entries, 0 to 17677336
Data columns (total 42 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Unnamed: 0          int64  
 1   timestamps_UTC      object 
 2   mapped_veh_id       float64
 3   lat                 float64
 4   lon                 float64
 5   RS_E_InAirTemp_PC1  float64
 6   RS_E_InAirTemp_PC2  float64
 7   RS_E_OilPress_PC1   float64
 8   RS_E_OilPress_PC2   float64
 9   RS_E_RPM_PC1        float64
 10  RS_E_RPM_PC2        float64
 11  RS_E_WatTemp_PC1    float64
 12  RS_E_WatTemp_PC2    float64
 13  RS_T_OilTemp_PC1    float64
 14  RS_T_OilTemp_PC2    float64
 15  date                object 
 16  hour                float64
 17  dayofweek           float64
 18  weekday             object 
 19  Distance            float64
 20  Speed               float64
 21  date_hour           object 
 22  datetime_x          object 
 23  weather_main        object 
 24  temp                fl

In [3]:
del data_info
del data_head
gc.collect()

53

In [4]:
def reduce_mem_usage(df):
    """ Iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if np.issubdtype(col_type, np.number):  # Check if column type is numeric
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type).startswith('int'):
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        # You can add more conditions here for other data types if necessary

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [5]:
# Apply the memory reduction
data = reduce_mem_usage(data)

Memory usage of dataframe is 5664.43 MB
Memory usage after optimization is: 2157.88 MB
Decreased by 61.9%


In [6]:
# data = data.drop(['Unnamed: 0', 'dayofweek', 'date_hour'], axis=1)

In [7]:
# data.head(500000).to_csv('mini_augumented_cleaned_ar41_for_ulb.csv', index=True)

# Status Label (running | stopped)

In [8]:
stopped_threshold = 10  # Speed less than 10 for being stopped
minimum_stopped_minutes = 30  # Minimum duration for stopped status

In [9]:
# Convert timestamps to datetime and sort
data['timestamps_UTC'] = pd.to_datetime(data['timestamps_UTC'])
data = data.sort_values('timestamps_UTC')

# Calculate time differences in minutes
data['time_diff'] = data['timestamps_UTC'].diff().dt.total_seconds() / 60.0

# Identify rows where the vehicle is stopped
data['is_stopped'] = data['Speed'] < stopped_threshold

# Forward-fill the 'is_stopped' status only within the groups where the vehicle is stopped
data['stopped_group'] = data['is_stopped'].ne(data['is_stopped'].shift()).cumsum()
data.loc[data['is_stopped'], 'stopped_group'] = data.loc[data['is_stopped'], 'stopped_group']

# Calculate the cumulative stopped time in minutes only for stopped groups
data['cumulative_stopped_time'] = data.groupby('stopped_group')['time_diff'].cumsum().fillna(0)

# Determine the stopped groups that exceed the minimum stopped duration
stopped_groups = data[data['cumulative_stopped_time'] > minimum_stopped_minutes]['stopped_group'].unique()

# Mark the status based on the identified stopped groups
data['status'] = 'running'
data.loc[data['stopped_group'].isin(stopped_groups), 'status'] = 'stopped'

In [10]:
data = data.dropna()

# Load the Models

In [11]:
# Load Isolation Forest model
iso_forest_model_path = '/kaggle/input/sncb-data-label-isolation-forest/isolation_forest_model.joblib'
iso_forest_model = joblib.load(iso_forest_model_path)

# Load LSTM Autoencoder model
lstm_autoencoder_model_path = '/kaggle/input/lstm-anomely-detection/lstm_autoencoder_model.h5'
lstm_autoencoder_model = load_model(lstm_autoencoder_model_path)

# Data Preprocessing Function

In [12]:
# Function to preprocess data
def preprocess_data(data, scaler=None, encoder=None):
    # Define numeric and categorical features
    numeric_features = [
        'RS_E_InAirTemp_PC1', 'RS_E_InAirTemp_PC2', 
        'RS_E_WatTemp_PC1', 'RS_E_WatTemp_PC2', 
        'RS_T_OilTemp_PC1', 'RS_T_OilTemp_PC2', 
        'Speed', 'lat', 'lon', 'mapped_veh_id', 
        'hour', 'pm10', 'temp_celsius', 'weekday'
    ]
    categorical_features = ['weather_main', 'status']

    # Convert 'weekday' to a numerical format
    data['weekday'] = pd.to_datetime(data['date']).dt.dayofweek

    # Standardize numeric features
    if scaler is None:
        scaler = StandardScaler()
        X_numeric_scaled = scaler.fit_transform(data[numeric_features])
    else:
        X_numeric_scaled = scaler.transform(data[numeric_features])

    # Encode categorical features
    if encoder is None:
        encoder = OneHotEncoder(sparse=False)
        X_categorical_encoded = encoder.fit_transform(data[categorical_features])
    else:
        X_categorical_encoded = encoder.transform(data[categorical_features])

    # Combine numeric and categorical data
    X_combined = np.hstack((X_numeric_scaled, X_categorical_encoded))
    
    return X_combined, scaler, encoder

In [13]:
# Preprocess the sample data for demonstration
X_combined, scaler, encoder = preprocess_data(data)
X_combined.shape

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(17480996, 25)

In [14]:
# pd.DataFrame(X_combined).head(50000).to_csv('X_combined_streaming.csv', index=True)

# Simulating Data Streaming

In [15]:
# Function to simulate data streaming
def stream_data(data, batch_size=100):
    for i in range(0, len(data), batch_size):
        # Yield a batch of data
        yield data[i:i + batch_size]

# Anomaly Detection in Streaming Mode

In [16]:
# # Function to detect anomalies
# def detect_anomalies(batch, iso_forest_model, lstm_autoencoder_model):
#     # Isolation Forest prediction
#     iso_forest_pred = iso_forest_model.predict(batch)

#     # LSTM Autoencoder prediction
#     # Reshape data for LSTM (assuming the model expects 3D input)
#     batch_reshaped = np.reshape(batch, (batch.shape[0], 1, batch.shape[1]))
#     reconstructed = lstm_autoencoder_model.predict(batch_reshaped)
#     mse = np.mean(np.power(batch_reshaped - reconstructed, 2), axis=1)
#     # A threshold for anomaly detection in LSTM Autoencoder (needs to be defined)
#     lstm_threshold = 0.5  # This is an example value
#     lstm_pred = mse > lstm_threshold

#     return iso_forest_pred, lstm_pred

In [49]:
def detect_anomalies(batch, iso_forest_model, lstm_autoencoder_model, iso_threshold=-0.1, lstm_threshold=2.5):
    # Reshape data for Isolation Forest (flatten the 3D batch to 2D)
    batch_flattened = batch.reshape(batch.shape[0] * batch.shape[1], -1)
    scores = iso_forest_model.decision_function(batch_flattened)
    iso_forest_anomalies = np.where(scores < iso_threshold)[0]  # Custom threshold

    # LSTM Autoencoder prediction (final step)
    reconstructed = lstm_autoencoder_model.predict(batch)
    actual_final_step = batch[:, -1, :]
    mse = np.mean(np.power(actual_final_step - reconstructed, 2), axis=1)
    lstm_pred = mse > lstm_threshold
    lstm_anomalies = np.where(lstm_pred)[0]

    return iso_forest_anomalies, lstm_anomalies

# Load Sample Data for Simulation

In [17]:
def batch_generator(data, sequence_length, batch_size, n_features):
    num_batches = int(np.ceil((data.shape[0] - sequence_length) / batch_size))
    while True:  # Loop forever so the generator never terminates
        for batch_index in range(num_batches):
            start_index = batch_index * batch_size
            end_index = start_index + sequence_length + batch_size - 1  # Adjusted end_index calculation
            
            # Ensure we do not go past the end of the data
            end_index = min(end_index, data.shape[0] - 1)
            
            batch_data = data[start_index:end_index]

            # Adjust batch size if we're at the end and have less than a full batch left
            current_batch_size = min(batch_size, data.shape[0] - start_index - sequence_length)
            X_batch = np.zeros((current_batch_size, sequence_length, n_features))
            y_batch = np.zeros((current_batch_size, n_features))

            for i in range(current_batch_size):
                end_seq_index = i + sequence_length
                # Make sure we don't go beyond the end of batch_data when creating the sequence
                if end_seq_index < batch_data.shape[0]:
                    X_batch[i] = batch_data[i:end_seq_index]
                    y_batch[i] = batch_data[end_seq_index]
                else:
                    # If not enough data for a full sequence, break the loop early
                    break

            yield X_batch, y_batch

In [23]:
n_features = X_combined.shape[1]  # Number of features

# Define sequence length (the window size) and batch size
sequence_length = 10  # This is a hyperparameter you can tune
batch_size = 1024 

# Initialize streaming data generator
streaming_data = batch_generator(X_combined, sequence_length, batch_size, n_features)

In [37]:
# Example of processing the data stream
for X_batch, _ in streaming_data:  # Unpack the tuple to get only the input data
    iso_forest_pred, lstm_pred = detect_anomalies(X_batch, iso_forest_model, lstm_autoencoder_model)
    # Process the results as needed (logging, alerts, etc.)
    print("Batch processed. Isolation Forest Anomalies:", np.sum(iso_forest_pred == -1),
          "LSTM Autoencoder Anomalies:", np.sum(lstm_pred))
    # Break after one batch for demonstration purposes
    break

32/32 [==============================] - 0s 5ms/step
Batch processed. Isolation Forest Anomalies: 0 LSTM Autoencoder Anomalies: 138101


In [51]:
# Process the data stream
for i, (X_batch, _) in enumerate(streaming_data):
    iso_forest_anomalies, lstm_anomalies = detect_anomalies(X_batch, iso_forest_model, lstm_autoencoder_model)
    print(f"Batch {i} processed. Isolation Forest Anomalies:", len(iso_forest_anomalies),
          "LSTM Autoencoder Anomalies:", len(lstm_anomalies))

    # Output specific information for anomalies
    if len(iso_forest_anomalies) > 0 or len(lstm_anomalies) > 0:
        # Assuming 'data' is the original DataFrame
        # Select the columns you want to display
        selected_columns = ['timestamps_UTC', 'lat', 'lon', 'RS_E_WatTemp_PC1', 'RS_T_OilTemp_PC1', 'RS_E_InAirTemp_PC1', 'Speed', 'status']
        anomaly_indices = np.unique(np.concatenate((iso_forest_anomalies, lstm_anomalies)))
        print("Anomaly details:")
        print(data.loc[anomaly_indices, selected_columns])

    # Break after a few batches for demonstration purposes
    if i >= 5:
        break

    # Delay for 2 seconds before the next batch
    time.sleep(2)

32/32 [==============================] - 0s 5ms/step
Batch 0 processed. Isolation Forest Anomalies: 0 LSTM Autoencoder Anomalies: 17
Anomaly details:
         timestamps_UTC       lat       lon  RS_E_WatTemp_PC1  \
26  2023-01-23 14:17:11  51.03125  3.769531              43.0   
123 2023-01-23 15:03:11  51.03125  3.769531              61.0   
219 2023-01-23 15:54:11  51.03125  3.710938              68.0   
222 2023-01-23 15:57:14  51.03125  3.699219              68.0   
282 2023-01-23 16:33:09  51.03125  3.720703              72.0   
291 2023-01-23 16:37:15  51.03125  3.759766              75.0   
322 2023-01-23 16:53:06  51.12500  3.689453              81.0   
331 2023-01-23 16:57:14  51.12500  3.640625              82.0   
415 2023-01-23 17:37:11  51.06250  3.740234              80.0   
425 2023-01-23 17:42:15  51.03125  3.730469              78.0   
505 2023-01-23 18:22:15  50.87500  3.630859              81.0   
508 2023-01-23 18:24:15  50.87500  3.619141              78.0   
602 2

# Save to CSV

In [ ]:
data.to_csv('labeled_augumented_cleaned_ar41_for_ulb.csv', index=True)